In [1]:
import os, re 
import numpy as np
import tensorflow as tf
import glob
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*' #os.getenv(x)함수는 환경 변수x의 값을 포함하는 문자열 변수를 반환합니다. txt_file_path 에 "/root/aiffel/lyricist/data/lyrics/*" 저장
txt_list = glob.glob(txt_file_path) #txt_file_path 경로에 있는 모든 파일명을 리스트 형식으로 txt_list 에 할당

raw_corpus = [] 

for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines() #read() : 파일 전체의 내용을 하나의 문자열로 읽어온다. , splitlines()  : 여러라인으로 구분되어 있는 문자열을 한라인씩 분리하여 리스트로 반환
        raw_corpus.extend(raw) # extend() : 리스트함수로 추가적인 내용을 연장 한다.

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])


데이터 크기: 187088
Examples:
 ["Busted flat in Baton Rouge, waitin' for a train", "And I's feelin' near as faded as my jeans", 'Bobby thumbed a diesel down, just before it rained']


In [2]:
# preprocess_sentence() 만들기
# len 15가 넘어가면 데이터에서 제외할것

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence


In [3]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

# raw_corpus list에 저장된 문장들을 순서대로 반환하여 sentence에 저장
for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    # 앞서 구현한 preprocess_sentence() 함수를 이용하여 문장을 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠

corpus[:10]

['<start> busted flat in baton rouge , waitin for a train <end>',
 '<start> and i s feelin near as faded as my jeans <end>',
 '<start> bobby thumbed a diesel down , just before it rained <end>',
 '<start> it rode us all the way to new orleans i pulled my harpoon out of my dirty red bandanna <end>',
 '<start> i was playin soft while bobby sang the blues , yeah <end>',
 '<start> windshield wipers slappin time , i was holdin bobby s hand in mine <end>',
 '<start> we sang every song that driver knew freedom s just another word for nothin left to lose <end>',
 '<start> nothin , don t mean nothin hon if it ain t free , no no <end>',
 '<start> and , feelin good was easy , lord , when he sang the blues <end>',
 '<start> you know , feelin good was good enough for me <end>']

In [4]:
def tokenize(corpus):

    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    
    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)   
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')
    
    print(tensor,tokenizer)
    return tensor, tokenizer



In [5]:
tensor, tokenizer = tokenize(corpus)

[[   2 3604 1691 ...    0    0    0]
 [   2    8    5 ...    0    0    0]
 [   2  804 7655 ...    0    0    0]
 ...
 [   2    5   22 ...    0    0    0]
 [   2    5   22 ...    0    0    0]
 [   2    5   22 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fe1a0126130>


In [6]:
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

src_input.shape

[   2 3604 1691   14 7050 3995    4 1100   28    9  681    3    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

(175749, 346)

In [7]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 128
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
 # tokenizer.num_words: 주어진 데이터의 문장들에서 빈도수가 높은 n개의 단어만 선택
 # tokenize() 함수에서 num_words를 7000개로 선언했기 때문에, tokenizer.num_words의 값은 7000
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((128, 346), (128, 346)), types: (tf.int32, tf.int32)>

In [8]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        # Embedding 레이어, 2개의 LSTM 레이어, 1개의 Dense 레이어로 구성되어 있다.
        # Embedding 레이어는 단어 사전의 인덱스 값을 해당 인덱스 번째의 워드 벡터로 바꿔준다.
        # 이 워드 벡터는 의미 벡터 공간에서 단어의 추상적 표현으로 사용된다. 
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size) 
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)  
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
# embedding size 값이 커질수록 단어의 추상적인 특징들을 더 잡아낼 수 있지만
# 그만큼 충분한 데이터가 없으면 안좋은 결과 값을 가져옵니다!   


In [9]:
embedding_size = 128 # 워드 벡터의 차원수를 말하며 단어가 추상적으로 표현되는 크기입니다.
hidden_size = 256 # 모델에 얼마나 많은 일꾼을 둘 것인가? 정도로 이해하면 좋다.
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size) # tokenizer.num_words에 +1인 이유는 문장에 없는 pad가 사용되었기 때문이다.


In [10]:
# train / test split
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state=34)


In [11]:
for src_sample, tgt_sample in dataset.take(1): break

model(src_sample)



<tf.Tensor: shape=(128, 346, 12001), dtype=float32, numpy=
array([[[-2.15208347e-05,  5.19308378e-05,  7.30644024e-05, ...,
          2.74989525e-05, -7.73094507e-05, -5.01392860e-05],
        [-6.86558051e-05,  4.62063945e-05,  5.15302781e-05, ...,
          1.08652792e-04, -2.46074691e-04, -1.01758553e-04],
        [-2.09704776e-05, -2.85966635e-05,  1.23270074e-05, ...,
          4.85017190e-05, -3.00845073e-04, -2.43695526e-04],
        ...,
        [ 3.01768398e-03,  6.59637735e-04,  9.54623451e-04, ...,
         -2.74755340e-03,  1.67235499e-03,  1.05832296e-03],
        [ 3.01768398e-03,  6.59637735e-04,  9.54623334e-04, ...,
         -2.74755340e-03,  1.67235523e-03,  1.05832284e-03],
        [ 3.01768398e-03,  6.59637735e-04,  9.54623451e-04, ...,
         -2.74755340e-03,  1.67235511e-03,  1.05832296e-03]],

       [[-2.15208347e-05,  5.19308378e-05,  7.30644024e-05, ...,
          2.74989525e-05, -7.73094507e-05, -5.01392860e-05],
        [-4.40205476e-05, -1.11773435e-04,  

In [12]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  1536128   
_________________________________________________________________
lstm (LSTM)                  multiple                  394240    
_________________________________________________________________
lstm_1 (LSTM)                multiple                  525312    
_________________________________________________________________
dense (Dense)                multiple                  3084257   
Total params: 5,539,937
Trainable params: 5,539,937
Non-trainable params: 0
_________________________________________________________________


In [13]:
optimizer = tf.keras.optimizers.Adam() # Adam은 현재 가장 많이 사용하는 옵티마이저이다. 자세한 내용은 차차 배운다.
loss = tf.keras.losses.SparseCategoricalCrossentropy( # 훈련 데이터의 라벨이 정수의 형태로 제공될 때 사용하는 손실함수이다.
    from_logits=True, # 기본값은 False이다. 모델에 의해 생성된 출력 값이 정규화되지 않았음을 손실 함수에 알려준다. 즉 softmax함수가 적용되지 않았다는걸 의미한다. 
    reduction='none'  # 기본값은 SUM이다. 각자 나오는 값의 반환 원할 때 None을 사용한다.
)
# 모델을 학습시키키 위한 학습과정을 설정하는 단계이다.
model.compile(loss=loss, optimizer=optimizer) # 손실함수와 훈련과정을 설정했다.

In [14]:
import tqdm
import tensorflow_addons as tfa

tqdm_callback = tfa.callbacks.TQDMProgressBar()

history = model.fit(dataset, epochs=10) # 만들어둔 데이터셋으로 모델을 학습한다. 30번 학습을 반복하겠다는 의미다.

Epoch 1/10
1373/1373 [==============================] - 586s 424ms/step - loss: 0.2399
Epoch 2/10
1373/1373 [==============================] - 589s 428ms/step - loss: 0.1466
Epoch 3/10
1373/1373 [==============================] - 590s 430ms/step - loss: 0.1378
Epoch 4/10
1373/1373 [==============================] - 595s 433ms/step - loss: 0.1314
Epoch 5/10
1373/1373 [==============================] - 596s 434ms/step - loss: 0.1260
Epoch 6/10
1373/1373 [==============================] - 600s 437ms/step - loss: 0.1210
Epoch 7/10
1373/1373 [==============================] - 601s 438ms/step - loss: 0.1164
Epoch 8/10
1373/1373 [==============================] - 602s 438ms/step - loss: 0.1123
Epoch 9/10
1373/1373 [==============================] - 605s 440ms/step - loss: 0.1085
Epoch 10/10
1373/1373 [==============================] - 602s 438ms/step - loss: 0.1051


In [15]:
#문장생성 함수 정의
#모델에게 시작 문장을 전달하면 모델이 시작 문장을 바탕으로 작문을 진행
def generate_text(model, tokenizer, init_sentence="<start>", max_len=15): #시작 문자열을 init_sentence 로 받으며 디폴트값은 <start> 를 받는다
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence]) #텍스트 안의 단어들을 숫자의 시퀀스의 형태로 변환
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다 (도달 하지 못하였으면 while 루프를 돌면서 다음 단어를 예측)
    while True: #루프를 돌면서 init_sentence에 단어를 하나씩 생성성
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4 
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated #최종적으로 모델이 생성한 문장을 반환

In [16]:
generate_text(model, tokenizer, init_sentence="<start> I love") # 시작문장으로 he를 넣어 문장생성 함수 실행

'<start> i love you , i love you , i love you , i love '

In [17]:
model_2 = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size) 

In [18]:
model_2(src_sample)

<tf.Tensor: shape=(128, 346, 12001), dtype=float32, numpy=
array([[[ 1.97829868e-05, -1.14249924e-04,  2.19506146e-05, ...,
         -9.30584501e-05, -1.28166022e-04,  4.87648940e-05],
        [ 8.88579507e-06,  2.18709101e-06,  5.89330339e-05, ...,
         -2.53088045e-04, -9.71993140e-05,  7.16876748e-05],
        [-2.13954377e-06,  4.78877628e-05, -9.50808608e-05, ...,
         -2.27371027e-04, -1.16590025e-04,  2.25399461e-04],
        ...,
        [ 4.19177022e-03, -2.25701858e-03,  1.48152700e-03, ...,
         -1.40247727e-03, -9.00637242e-04, -8.89614341e-04],
        [ 4.19176975e-03, -2.25701788e-03,  1.48152700e-03, ...,
         -1.40247727e-03, -9.00636951e-04, -8.89614341e-04],
        [ 4.19176975e-03, -2.25701788e-03,  1.48152700e-03, ...,
         -1.40247727e-03, -9.00637009e-04, -8.89614224e-04]],

       [[ 1.97829868e-05, -1.14249924e-04,  2.19506146e-05, ...,
         -9.30584501e-05, -1.28166022e-04,  4.87648940e-05],
        [-7.54138237e-05, -1.65447073e-05,  

In [19]:
model_2.summary()

Model: "text_generator_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      multiple                  1536128   
_________________________________________________________________
lstm_2 (LSTM)                multiple                  394240    
_________________________________________________________________
lstm_3 (LSTM)                multiple                  525312    
_________________________________________________________________
dense_1 (Dense)              multiple                  3084257   
Total params: 5,539,937
Trainable params: 5,539,937
Non-trainable params: 0
_________________________________________________________________


In [20]:
model_2.compile(loss=loss, optimizer=optimizer)

In [21]:
tqdm_callback2 = tfa.callbacks.TQDMProgressBar()

history2 = model.fit(enc_train, dec_train, epochs=10, batch_size=BATCH_SIZE, validation_data=(enc_val, dec_val))

Epoch 1/10
1099/1099 [==============================] - 566s 514ms/step - loss: 0.1019 - val_loss: 0.1022
Epoch 2/10
1099/1099 [==============================] - 564s 513ms/step - loss: 0.0990 - val_loss: 0.1025
Epoch 3/10
1099/1099 [==============================] - 562s 511ms/step - loss: 0.0964 - val_loss: 0.1025
Epoch 4/10
1099/1099 [==============================] - 561s 511ms/step - loss: 0.0940 - val_loss: 0.1025
Epoch 5/10
1099/1099 [==============================] - 563s 513ms/step - loss: 0.0918 - val_loss: 0.1027
Epoch 6/10
1099/1099 [==============================] - 565s 514ms/step - loss: 0.0897 - val_loss: 0.1027
Epoch 7/10
1099/1099 [==============================] - 567s 516ms/step - loss: 0.0878 - val_loss: 0.1029
Epoch 8/10
1099/1099 [==============================] - 567s 516ms/step - loss: 0.0859 - val_loss: 0.1032
Epoch 9/10
1099/1099 [==============================] - 568s 517ms/step - loss: 0.0842 - val_loss: 0.1034
Epoch 10/10
1099/1099 [=======================

In [22]:
generate_text(model_2, tokenizer, init_sentence="<start> I love") # 시작문장으로 he를 넣어 문장생성 함수 실행

'<start> i love muddy muddy grippin grippin grippin fatigues fatigues fatigues parkin parkin bitter pompous '

In [ ]:
이번 노드는 경석님과 함꼐 진행 했숩니다.